<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### 第六週 第四天

現在介紹本週的重點專案：


# 自主交易員

這是一個股票交易模擬，有 4 位交易員和 1 位研究員，透過多個 MCP 伺服器提供工具與資源：

1. 我們自製的 Accounts MCP 伺服器（由工程團隊開發！）
2. Fetch（使用本地 headless browser 抓取網頁）
3. 記憶體
4. Brave 搜尋
5. 財經資料

還有一個資源可以讀取交易員的帳戶資訊與投資策略。

今天的 Lab 目標是建立一個新的 python 模組 `traders.py`，用來管理交易場上的單一交易員。

我們會在 Lab 中先實驗與探索，等準備好再移植到 python 模組。


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [ ]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

### Let's start by gathering the MCP params for our trader

In [ ]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

In [13]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [ ]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""你是一位金融研究員。你能夠在網路上搜尋有趣的財經新聞，
尋找可能的交易機會，並協助進行研究。
根據請求，你會執行必要的研究並回覆你的發現。
請多花點時間進行多次搜尋，以獲得全面的資訊，然後總結你的發現。
如果沒有特定請求，請根據最新新聞搜尋，回覆投資機會。
目前的日期時間是 {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""

    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [ ]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="這個工具會在網路上搜尋新聞與投資機會，\
                可以根據你的特定請求調查某支股票，\
                或是一般性地搜尋重要的財經新聞與投資機會。\
                請描述你想要的研究方向。"
        )

In [ ]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



### Look at the trace

https://platform.openai.com/traces

In [ ]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

### And now - to create our Trader Agent

In [ ]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
你是一位管理股票投資組合的交易員。你的名字是 {agent_name}，你的帳戶也以 {agent_name} 為名。
你可以使用工具來搜尋公司新聞、查詢股價，以及買賣股票。
你的投資組合策略如下：
{strategy}
你目前的持股與餘額如下：
{account_details}
你有工具可以進行網路搜尋，取得相關新聞與資訊。
你有工具可以查詢股價。
你有工具可以買賣股票。
你有工具可以記錄公司、研究與目前思考的內容。
請善用這些工具管理你的投資組合。請依照你的判斷執行交易，不需等待指示或確認。
"""

prompt = """
請使用你的工具來決定投資組合的操作。
調查新聞與市場狀況，做出決策，執行交易，並回覆你的行動摘要。
"""

In [ ]:
print(instructions)

### And to run our Trader

In [ ]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Then go and look at the trace

http://platform.openai.com/traces


In [ ]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

### 現在來檢視由此產生的 Python 模組：

`mcp_params.py` 用來指定 MCP 伺服器。你會發現我加入了一些熟悉的朋友：記憶體與推播通知！

`templates.py` 用來設定指令與訊息（也就是 System prompt 與 User prompt）

`traders.py` 則是將所有內容整合在一起。

你會注意到我用了一個比較精巧的寫法：

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

這只是把多個 "with" 敘述（也就是 context manager）合併成一個乾淨的寫法，這樣就不用寫得很醜：

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

但本質上是等價的。


In [2]:
from traders import Trader


In [3]:
trader = Trader("Ed")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")